# The Plastic Tide: Forecasting Macroplastic Pollution in Major River Systems

This notebook implements the full pipeline to predict riverine plastic waste outflow from socioeconomic, geographic, and infrastructural features.

- Data sources: Plastic River Footprints, World Bank, GRID3.
- Models: Ridge, Lasso, Gradient Boosting (XGBoost).
- Metrics: R-squared (R²), MAPE.

Run cells top to bottom. See `report/` for LaTeX report template.

In [ ]:
import os
import json
import math
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_percentage_error

from xgboost import XGBRegressor

warnings.simplefilter('ignore')

PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

print('Project root:', PROJECT_ROOT)
print('Data dir:', DATA_DIR)
print('Outputs dir:', OUTPUTS_DIR)

## Data loading

Provide CSV files in `data/` directory:
- `rivers_plastic.csv`: target `plastic_kg_per_year` and river identifiers
- `world_bank.csv`: socioeconomic indicators per country
- `grid3_population.csv`: population density or totals near river buffers

You may adapt filenames/columns below to your specific datasets.

In [ ]:
# File paths (adjust as needed)
rivers_fp = DATA_DIR / 'rivers_plastic.csv'
world_bank_fp = DATA_DIR / 'world_bank.csv'
grid3_fp = DATA_DIR / 'grid3_population.csv'

# Load with safe defaults
rivers_df = pd.read_csv(rivers_fp) if rivers_fp.exists() else pd.DataFrame()
wb_df = pd.read_csv(world_bank_fp) if world_bank_fp.exists() else pd.DataFrame()
grid_df = pd.read_csv(grid3_fp) if grid3_fp.exists() else pd.DataFrame()

print('rivers_df shape:', rivers_df.shape)
print('wb_df shape:', wb_df.shape)
print('grid_df shape:', grid_df.shape)

# Basic sanity checks
expected_target = 'plastic_kg_per_year'
if not rivers_df.empty and expected_target not in rivers_df.columns:
    raise ValueError(f"Expected target column '{expected_target}' missing in rivers_plastic.csv")

## Data merging and feature engineering

We merge river-level target data with country-level indicators and local population features. Replace keys as appropriate for your data (e.g., `country`, `river_id`).

In [ ]:
def safe_lower(series):
    return series.astype(str).str.lower().str.strip()

if not rivers_df.empty:
    # Standardize join keys
    if 'country' in rivers_df.columns:
        rivers_df['country_key'] = safe_lower(rivers_df['country'])
    if 'river_id' not in rivers_df.columns:
        rivers_df['river_id'] = np.arange(len(rivers_df))

if not wb_df.empty:
    # Expect a country column
    key_col = 'country' if 'country' in wb_df.columns else ('Country' if 'Country' in wb_df.columns else None)
    if key_col:
        wb_df['country_key'] = safe_lower(wb_df[key_col])

if not grid_df.empty:
    # Expect river_id or a join key present in rivers
    if 'river_id' not in grid_df.columns and 'river_name' in grid_df.columns and 'river_name' in rivers_df.columns:
        grid_df = grid_df.merge(
            rivers_df[['river_id', 'river_name']].assign(river_name_key=safe_lower(rivers_df['river_name'])),
            left_on=grid_df['river_name'].str.lower().str.strip(),
            right_on='river_name_key',
            how='left'
        )

# Merge
df = rivers_df.copy()
if not wb_df.empty and 'country_key' in df.columns and 'country_key' in wb_df.columns:
    wb_cols = [c for c in wb_df.columns if c not in ['country', 'Country']]
    df = df.merge(wb_df[wb_cols], on='country_key', how='left')

if not grid_df.empty:
    join_cols = ['river_id'] if 'river_id' in grid_df.columns else []
    if join_cols:
        df = df.merge(grid_df, on=join_cols, how='left')

print('Merged df shape:', df.shape)
print('Columns:', list(df.columns)[:20], '...')

## Train/validation/test split and preprocessing

We select numeric features, impute missing values, and scale for linear models. Tree models handle scaling implicitly.

In [ ]:
# Select target and features
TARGET_COL = 'plastic_kg_per_year'
if df.empty or TARGET_COL not in df.columns:
    print('Dataset not ready. Please place CSVs in data/ and rerun.')
else:
    y = df[TARGET_COL].astype(float)
    # Simple numeric-only baseline, drop identifiers and text columns
    drop_like = {'river_name', 'country', 'country_key'}
    numeric_cols = [c for c in df.columns if c not in drop_like and c != TARGET_COL and pd.api.types.is_numeric_dtype(df[c])]
    X = df[numeric_cols].copy()

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    print('Train/Valid/Test:', X_train.shape, X_valid.shape, X_test.shape)

    # Preprocessing for linear models
    numeric_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer(
        transformers=[('num', numeric_pipeline, numeric_cols)],
        remainder='drop'
    )

    # Baseline Ridge
    ridge = Pipeline(steps=[('prep', preprocessor), ('model', Ridge())])
    ridge_param_grid = {'model__alpha': [0.1, 1.0, 10.0, 100.0]}

    ridge_cv = GridSearchCV(ridge, ridge_param_grid, scoring='r2', cv=5, n_jobs=-1)
    ridge_cv.fit(X_train, y_train)
    y_pred = ridge_cv.predict(X_valid)
    print('Ridge best alpha:', ridge_cv.best_params_['model__alpha'])
    print('Ridge valid R2:', r2_score(y_valid, y_pred))

    # Lasso
    lasso = Pipeline(steps=[('prep', preprocessor), ('model', Lasso(max_iter=5000))])
    lasso_param_grid = {'model__alpha': [0.001, 0.01, 0.1, 1.0]}
    lasso_cv = GridSearchCV(lasso, lasso_param_grid, scoring='r2', cv=5, n_jobs=-1)
    lasso_cv.fit(X_train, y_train)
    y_pred = lasso_cv.predict(X_valid)
    print('Lasso best alpha:', lasso_cv.best_params_['model__alpha'])
    print('Lasso valid R2:', r2_score(y_valid, y_pred))

    # XGBoost
    xgb = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective='reg:squarederror',
        n_jobs=-1,
    )
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_valid)
    print('XGB valid R2:', r2_score(y_valid, y_pred))

    # Choose best by valid R2
    models = {
        'ridge': ridge_cv.best_estimator_,
        'lasso': lasso_cv.best_estimator_,
        'xgb': xgb,
    }
    scores = {name: r2_score(y_valid, (m.predict(X_valid))) for name, m in models.items()}
    best_name = max(scores, key=scores.get)
    best_model = models[best_name]
    print('Best model:', best_name, 'R2:', scores[best_name])

    # Final test evaluation
    y_pred_test = best_model.predict(X_test)
    r2 = r2_score(y_test, y_pred_test)
    mape = mean_absolute_percentage_error(y_test, y_pred_test)
    print('Test R2:', r2)
    print('Test MAPE:', mape)

    # Save metrics
    metrics = {'valid_scores': scores, 'test_r2': float(r2), 'test_mape': float(mape), 'best_model': best_name, 'features': numeric_cols}
    (OUTPUTS_DIR / 'metrics.json').write_text(json.dumps(metrics, indent=2))

## Plots: Parity and Residuals

In [ ]:
if df.empty or TARGET_COL not in df.columns:
    pass
else:
    def parity_plot(y_true, y_pred, title, fname):
        plt.figure(figsize=(5,5))
        lims = [min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())]
        plt.scatter(y_true, y_pred, alpha=0.6)
        plt.plot(lims, lims, 'r--')
        plt.xlabel('True')
        plt.ylabel('Predicted')
        plt.title(title)
        plt.tight_layout()
        plt.savefig(OUTPUTS_DIR / fname, dpi=150)
        plt.show()

    # Using validation set predictions from best model
    y_pred_valid = best_model.predict(X_valid)
    parity_plot(y_valid, y_pred_valid, f'Parity plot ({best_name})', 'parity_valid.png')

    residuals = y_valid - y_pred_valid
    plt.figure(figsize=(6,4))
    sns.histplot(residuals, kde=True)
    plt.title('Residuals distribution (validation)')
    plt.tight_layout()
    plt.savefig(OUTPUTS_DIR / 'residuals_valid.png', dpi=150)
    plt.show()

## Feature importance (XGBoost)

We report gain-based importances to inform policy-relevant drivers.

In [ ]:
if df.empty or TARGET_COL not in df.columns:
    pass
else:
    if isinstance(best_model, XGBRegressor):
        importances = best_model.feature_importances_
        fi = pd.DataFrame({'feature': X_train.columns, 'importance': importances}).sort_values('importance', ascending=False)
        fi.head(20).to_csv(OUTPUTS_DIR / 'feature_importance_top20.csv', index=False)
        plt.figure(figsize=(8,6))
        sns.barplot(x='importance', y='feature', data=fi.head(20))
        plt.title('Top 20 feature importances (XGB)')
        plt.tight_layout()
        plt.savefig(OUTPUTS_DIR / 'feature_importance_top20.png', dpi=150)
        plt.show()
    else:
        print('Best model is not XGB; skipping importance plot.')

## Save model (optional)

Persist the best model for later inference.

In [ ]:
import pickle
if not (df.empty or TARGET_COL not in df.columns):
    with open(OUTPUTS_DIR / f'model_{best_name}.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print('Saved model to:', OUTPUTS_DIR / f'model_{best_name}.pkl')